[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LGM-jupyter/blob/main/LGM_ply_to_glb_jupyter.ipynb)

In [ ]:
%cd /content
!git clone --recursive -b dev https://github.com/camenduru/LGM
%cd /content/LGM

!pip install -q tyro mcubes PyMCubes nerfacc nvdiffrast trimesh pymeshlab ninja
!pip install -q https://github.com/camenduru/wheels/releases/download/colab/nvdiffrast-0.3.1-py3-none-any.whl
!pip install -q git+https://github.com/graphdeco-inria/diff-gaussian-rasterization
!pip install -q git+https://github.com/ashawkey/kiuikit.git

In [ ]:
%cd /content/LGM
!python convert.py